In [1]:
####
#Title: Stock_Model_LSTM
#Author: Guo Zebin

#Description: This is stock prediction model using ML models
#             Use LSTM and multiple features
#Evaluation: Using MSE
#Conclusion: LSTM performs far more better than lr and tree
#Discussion: 1. parameter adjustment? 2. does these features reasonable as training data?

In [2]:
#Import all the library used
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

In [ ]:
#load the data from local environment
from google.colab import files
uploaded = files.upload()

In [ ]:
#store the data into df
df = pd.read_csv('netflix.csv')
df.tail(1000)

In [5]:
# get the size of data
df.shape

(4881, 7)

In [ ]:
#visualise the close price data
plt.figure(figsize=(16,8))
plt.title('Netflix Close Price Data')
plt.xlabel('Days')
plt.ylabel('close price')
plt.plot(df['Close'][4000:])
plt.show()

In [ ]:
# Get training part(Why choose 2018-01-01?)
df_training = df[df['Date'] < '2018-01-01']
df_training

In [ ]:
# Get testing part
df_testing = df[df['Date'] >= '2018-01-01']
df_testing

In [ ]:
# get rid of trashy features(Why drop adj close?idk)
Adj_training = df_training.drop(['Date','Adj Close'],1)
Adj_training.head()

In [ ]:
# get rid of trashy features(Why drop adj close?)
Adj_testing = df_testing.drop(['Date','Adj Close'],1)
Adj_testing.shape

In [ ]:
# Feature scaling
scaler = MinMaxScaler()
Adj_training = scaler.fit_transform(Adj_training)
Adj_testing = scaler.fit_transform(Adj_testing)
Adj_testing

In [40]:
X_train = []
y_train = []
X_test = []
y_test = []

In [ ]:
Adj_training.shape

In [42]:
# Create training set
for i in range(60, Adj_training.shape[0]):
  X_train.append(Adj_training[i-60:i])
  y_train.append(Adj_training[i, 0])

In [46]:
X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
print(X_train.shape)

In [43]:
# Create testing set
for i in range(60, Adj_testing.shape[0]):
  X_test.append(Adj_testing[i-60:i])
  y_test.append(Adj_testing[i, 0])

In [ ]:
X_test, y_test = np.array(X_test), np.array(y_test)
X_test
y_test

In [80]:
# create LSTM model(figure out what each parameters are doing IMPORTANT!)
model = Sequential()

model.add(LSTM(units = 50, activation='relu', return_sequences = True, input_shape = (X_train.shape[1], 5)))
model.add(Dropout(0.2))
model.add(LSTM(units = 60, activation='relu', return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 60, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units = 1))

In [ ]:
model.summary()

In [ ]:
#Now do the trainging(this is one of the judgement of parameter adjustment)
model.compile(optimizer='adam', loss = 'mean_squared_error')
model.fit(X_train,y_train,epochs = 20,batch_size = 32)

In [ ]:
X_test.shape

In [ ]:
# Do the prediction
y_predict = model.predict(X_test)
y_predict[100]

In [ ]:
##visualise the data
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Days')
plt.ylabel('predict price')
plt.plot(y_test, color = 'red', label = 'Real price')
plt.plot(y_predict, color = 'green', label = 'Predicted price')
plt.legend()
plt.style.use('bmh')
plt.show()

In [ ]:
print('MSE:')
print(mean_squared_error(y_test, y_predict))